In [1]:
import os
import yaml
from torch.optim import SGD
from torch.optim.lr_scheduler import StepLR
from torch.nn import CrossEntropyLoss

from utils import (
    load_model_input_data,
    generate_evaluation_report,
    create_checkpoints,
    print_training_summary
)

from data_preparation import split_data_by_proportions
from data_setup import create_dataloaders
from engine import train_and_validate_model

from r2plus1d_network import R2Plus1DConvNet
from callbacks import InitStopper, EarlyStopper, ModelCheckpoint
from accuracy import BinaryAccuracy

In [2]:
with open("configs.yml", 'r') as file:
    parameters = yaml.safe_load(file)

data_params = parameters["data_parameters"]
model_params = parameters["model_parameters"]

In [3]:
main_directory = os.path.dirname(os.getcwd())
data_path = os.path.join(main_directory, "data")
primary_data_path = os.path.join(data_path, "primary")
model_directory_path = os.path.join(data_path, "models")

In [4]:
input_data = load_model_input_data(primary_data_path, parameters)

In [5]:
data_splits = split_data_by_proportions(input_data, data_params)

train_data, valid_data, test_data = create_dataloaders(data_splits, data_params)

In [6]:
model = R2Plus1DConvNet(
    in_channels=model_params["in_channels"],
    num_classes=model_params["num_classes"],
    dropout=model_params["dropout"])

model = model.to(model_params["device"])

In [7]:
loss_fn = CrossEntropyLoss()
accuracy_fn = BinaryAccuracy()

optimizer = SGD(
    params=model.parameters(),
    lr=model_params["learning_rate"],
    weight_decay=model_params["weight_decay"])
    
lr_scheduler = StepLR(
    optimizer,
    step_size=model_params["step_size"],
    gamma=model_params["gamma"])

In [8]:
init_stopper = InitStopper(patience=model_params["init_stopper_patience"])

early_stopper = EarlyStopper(
    patience=model_params["early_stopper_patience"],
    min_delta=model_params["early_stopper_min_delta"])

model_checkpoint = ModelCheckpoint(maximize=model_params["maximize"])

In [9]:
results = train_and_validate_model(
    model=model,
    train_dataloader=train_data,
    valid_dataloader=valid_data,
    loss_fn=loss_fn,
    accuracy_fn=accuracy_fn,
    optimizer=optimizer,
    lr_scheduler=lr_scheduler,
    model_checkpoint=model_checkpoint,
    num_epochs=model_params["num_epochs"])

Training has started...


Epoch: 1
Training loss: 0.6953 | Validation loss: 0.6929
Training accuracy: 0.4965 | Validation accuracy: 0.5161


Epoch: 2
Training loss: 0.6942 | Validation loss: 0.6929
Training accuracy: 0.5018 | Validation accuracy: 0.5161


Epoch: 3
Training loss: 0.6947 | Validation loss: 0.6929
Training accuracy: 0.4992 | Validation accuracy: 0.5161


Epoch: 4
Training loss: 0.6942 | Validation loss: 0.6927
Training accuracy: 0.5023 | Validation accuracy: 0.5161


Epoch: 5
Training loss: 0.6957 | Validation loss: 0.6928
Training accuracy: 0.4868 | Validation accuracy: 0.5161


Epoch: 6
Training loss: 0.6949 | Validation loss: 0.6929
Training accuracy: 0.4857 | Validation accuracy: 0.5161


Epoch: 7
Training loss: 0.6950 | Validation loss: 0.6929
Training accuracy: 0.4850 | Validation accuracy: 0.5161


Epoch: 8
Training loss: 0.6945 | Validation loss: 0.6928
Training accuracy: 0.4924 | Validation accuracy: 0.5161


Epoch: 9
Training loss: 0.6937 | Validation loss: 0.69

In [10]:
best_training_step = model_checkpoint.get_best_training_step()

checkpoints = create_checkpoints(
    optimizer, lr_scheduler, results, best_training_step)

In [11]:
print_training_summary(results)

-- -- Training Summary: -- --
Number of Epochs: 10

Final Training Loss: 0.6945
Final Validation Loss: 0.6927

Final Training Accuracy: 0.4990
Final Validation Accuracy: 0.5161



In [12]:
evaluation_report = generate_evaluation_report(
    model, test_data, checkpoints["best_training_step"], model_params)

-- -- Model Evaluation -- --
-- Evaluation Stage: final --
Loss: 0.6934
Accuracy: 0.5029

Confusion Matrix:
 [[  0 343]
 [  0 347]]

Precision Score: 0.5029
Recall Score: 1.0000
F1 Score: 0.6692

